In [27]:
import pandas as pd
import numpy as np
import os
import glob
import json
import time
from collections import defaultdict
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import xgboost as xgb

## Configuration

In [2]:
# Number of force cells in the robotic leg
N_CELLS = 8

# Path where the results are stored
RESULTS_PATH = '../../../results'
# ID of the training and test data resulting from this notebook, stored in RESULTS_PATH
DATA_ID = '0004_15042021'
# Hyperparameters search date
HS_DATE = '15042021'
# Number of folds in cross-validation
CV = 6

print('Model trained with data: ' + DATA_ID)

pd.set_option('display.max_columns', None)

Model trained with data: 0004_15042021


## Hyperparameters seach analysis

In [18]:
results_files_ls = glob.glob(os.path.join(RESULTS_PATH, DATA_ID, 'XGB_{}'.format(HS_DATE), 'XGB_{}_*.json'.format(HS_DATE)))

print('Number of results files: {}'.format(len(results_files_ls)))

Number of results files: 486


In [19]:
# Load all the results and generates a pandas dataframe
results_ls = []
for results_file in results_files_ls:
    with open(results_file) as json_file:
        results_dict = json.load(json_file)
        
    dict_aux = {}
    dict_aux['params_ID'] = results_dict['id']
    for key, value in results_dict['parameters'].items():
        dict_aux['param_' + key] = value
    for key, value in results_dict['cv_results'].items():
        dict_aux['__'.join([key, 'mean'])] = np.mean(value)
        dict_aux['__'.join([key, 'std'])] = np.std(value)

    results_ls.append(dict_aux)
        
results_df = pd.DataFrame(results_ls)
results_df

,params_ID,param_booster,param_eta,param_gamma,param_lambda,param_max_depth,param_nthread,param_objective,param_seed,param_subsample,fit_time__mean,fit_time__std,Train_Fx_MAE_mean__mean,Train_Fx_MAE_mean__std,Train_Fx_MAE_std__mean,Train_Fx_MAE_std__std,Train_Fx_MSE_mean__mean,Train_Fx_MSE_mean__std,Train_Fx_MSE_std__mean,Train_Fx_MSE_std__std,Train_Fx_R2_mean__mean,Train_Fx_R2_mean__std,Train_Fx_R2_std__mean,Train_Fx_R2_std__std,Train_Fy_MAE_mean__mean,Train_Fy_MAE_mean__std,Train_Fy_MAE_std__mean,Train_Fy_MAE_std__std,Train_Fy_MSE_mean__mean,Train_Fy_MSE_mean__std,Train_Fy_MSE_std__mean,Train_Fy_MSE_std__std,Train_Fy_R2_mean__mean,Train_Fy_R2_mean__std,Train_Fy_R2_std__mean,Train_Fy_R2_std__std,Train_Fz_MAE_mean__mean,Train_Fz_MAE_mean__std,Train_Fz_MAE_std__mean,Train_Fz_MAE_std__std,Train_Fz_MSE_mean__mean,Train_Fz_MSE_mean__std,Train_Fz_MSE_std__mean,Train_Fz_MSE_std__std,Train_Fz_R2_mean__mean,Train_Fz_R2_mean__std,Train_Fz_R2_std__mean,Train_Fz_R2_std__std,Test_Fx_MAE_mean__mean,Test_Fx_MAE_mean__std,Test_Fx_MAE_std__mean,Test_Fx_MAE_std__std,Test_Fx_MSE_mean__mean,Test_Fx_MSE_mean__std,Test_Fx_MSE_std__mean,Test_Fx_MSE_std__std,Test_Fx_R2_mean__mean,Test_Fx_R2_mean__std,Test_Fx_R2_std__mean,Test_Fx_R2_std__std,Test_Fy_MAE_mean__mean,Test_Fy_MAE_mean__std,Test_Fy_MAE_std__mean,Test_Fy_MAE_std__std,Test_Fy_MSE_mean__mean,Test_Fy_MSE_mean__std,Test_Fy_MSE_std__mean,Test_Fy_MSE_std__std,Test_Fy_R2_mean__mean,Test_Fy_R2_mean__std,Test_Fy_R2_std__mean,Test_Fy_R2_std__std,Test_Fz_MAE_mean__mean,Test_Fz_MAE_mean__std,Test_Fz_MAE_std__mean,Test_Fz_MAE_std__std,Test_Fz_MSE_mean__mean,Test_Fz_MSE_mean__std,Test_Fz_MSE_std__mean,Test_Fz_MSE_std__std,Test_Fz_R2_mean__mean,Test_Fz_R2_mean__std,Test_Fz_R2_std__mean,Test_Fz_R2_std__std
0,PTEHVO2QK2,gbtree,0.3,0.10,1.2,12,8,reg:squarederror,0,0.50,20.806981,1.300716,5.828721,2.136396,3.286920,1.993901,85.646589,53.755825,87.850878,79.804586,0.761662,0.124334,0.194398,0.117103,7.352057,1.538535,6.037452,1.380673,184.165590,69.376084,248.667178,95.489092,0.621482,0.130884,0.181017,0.047947,6.712153,3.118217,3.379021,2.111139,102.311511,86.018884,90.420212,94.483530,0.798421,0.156355,0.192776,0.178722,12.126970,1.637808,4.012353,1.039281,310.874458,82.935998,209.782153,98.922784,-0.072073,0.380281,0.312984,0.223286,11.403196,1.550200,8.376759,1.883289,425.733335,111.011080,630.313615,171.386303,-0.073392,0.116810,0.261770,0.101672,16.126198,2.805009,6.517967,1.687439,621.968472,185.555494,581.575327,281.867730,-0.322329,0.547673,0.516527,0.196665
1,GOSEIOZHXE,gbtree,0.2,0.01,1.2,12,8,reg:squarederror,0,0.50,21.770642,1.148223,6.401059,1.743584,3.026555,1.196900,88.854627,43.929458,75.520205,45.144954,0.750736,0.112639,0.168578,0.097841,7.423994,1.595177,6.057679,1.760923,189.963198,88.618753,263.647299,138.762248,0.606946,0.142093,0.216146,0.055919,8.358155,2.304446,3.608157,1.885310,130.517789,72.982303,100.868721,92.081807,0.752268,0.150327,0.206507,0.198816,12.235199,1.485155,4.281006,0.938203,312.228844,74.970537,218.321190,88.823604,-0.072885,0.366872,0.325108,0.220172,11.250470,1.586538,8.324505,1.924200,417.348033,110.528910,628.799949,167.527628,-0.038149,0.107098,0.231810,0.073191,16.584548,2.835792,6.845719,1.453005,648.212614,185.035502,618.586435,307.224457,-0.371530,0.541521,0.559801,0.236390
2,BU8XOLH8WK,gbtree,0.4,0.01,0.8,6,8,reg:squarederror,0,0.50,9.952336,0.566039,7.723353,1.604774,3.391554,1.965806,131.527493,55.185971,111.501712,92.260395,0.647833,0.112687,0.184898,0.095764,8.207370,0.765423,6.064603,0.719504,208.226473,34.601198,261.998117,46.700908,0.539282,0.079351,0.167955,0.030269,9.089048,2.050127,3.987035,2.084723,178.702493,79.996708,134.036324,104.320017,0.666784,0.148610,0.249400,0.192878,11.967705,1.714449,3.928991,1.161412,297.127454,79.650819,194.414553,86.228322,-0.026313,0.351857,0.316866,0.193391,11.203747,1.631814,8.276788,1.677309,413.655956,110.204796,608.670352,163.582742,-0.046018,0.170482,0.311183,0.292065,15.970900,3.174752,6.421824,1.778102,610

In [20]:
# Sum up the scores by force axis in only one sortable score
for subset in ['Train', 'Test']:
    for loss in ['MAE', 'MSE', 'R2']:
        results_df[subset + '_' + loss] = results_df[[subset + '_' + force + '_' + loss + '_mean__mean' for force in ['Fx', 'Fy', 'Fz']]].mean(axis=1)

In [21]:
# Sort the dataframe by the most relevant score
results_df = results_df.sort_values(['Test_R2'], ascending=False)
results_df

,params_ID,param_booster,param_eta,param_gamma,param_lambda,param_max_depth,param_nthread,param_objective,param_seed,param_subsample,fit_time__mean,fit_time__std,Train_Fx_MAE_mean__mean,Train_Fx_MAE_mean__std,Train_Fx_MAE_std__mean,Train_Fx_MAE_std__std,Train_Fx_MSE_mean__mean,Train_Fx_MSE_mean__std,Train_Fx_MSE_std__mean,Train_Fx_MSE_std__std,Train_Fx_R2_mean__mean,Train_Fx_R2_mean__std,Train_Fx_R2_std__mean,Train_Fx_R2_std__std,Train_Fy_MAE_mean__mean,Train_Fy_MAE_mean__std,Train_Fy_MAE_std__mean,Train_Fy_MAE_std__std,Train_Fy_MSE_mean__mean,Train_Fy_MSE_mean__std,Train_Fy_MSE_std__mean,Train_Fy_MSE_std__std,Train_Fy_R2_mean__mean,Train_Fy_R2_mean__std,Train_Fy_R2_std__mean,Train_Fy_R2_std__std,Train_Fz_MAE_mean__mean,Train_Fz_MAE_mean__std,Train_Fz_MAE_std__mean,Train_Fz_MAE_std__std,Train_Fz_MSE_mean__mean,Train_Fz_MSE_mean__std,Train_Fz_MSE_std__mean,Train_Fz_MSE_std__std,Train_Fz_R2_mean__mean,Train_Fz_R2_mean__std,Train_Fz_R2_std__mean,Train_Fz_R2_std__std,Test_Fx_MAE_mean__mean,Test_Fx_MAE_mean__std,Test_Fx_MAE_std__mean,Test_Fx_MAE_std__std,Test_Fx_MSE_mean__mean,Test_Fx_MSE_mean__std,Test_Fx_MSE_std__mean,Test_Fx_MSE_std__std,Test_Fx_R2_mean__mean,Test_Fx_R2_mean__std,Test_Fx_R2_std__mean,Test_Fx_R2_std__std,Test_Fy_MAE_mean__mean,Test_Fy_MAE_mean__std,Test_Fy_MAE_std__mean,Test_Fy_MAE_std__std,Test_Fy_MSE_mean__mean,Test_Fy_MSE_mean__std,Test_Fy_MSE_std__mean,Test_Fy_MSE_std__std,Test_Fy_R2_mean__mean,Test_Fy_R2_mean__std,Test_Fy_R2_std__mean,Test_Fy_R2_std__std,Test_Fz_MAE_mean__mean,Test_Fz_MAE_mean__std,Test_Fz_MAE_std__mean,Test_Fz_MAE_std__std,Test_Fz_MSE_mean__mean,Test_Fz_MSE_mean__std,Test_Fz_MSE_std__mean,Test_Fz_MSE_std__std,Test_Fz_R2_mean__mean,Test_Fz_R2_mean__std,Test_Fz_R2_std__mean,Test_Fz_R2_std__std,Train_MAE,Train_MSE,Train_R2,Test_MAE,Test_MSE,Test_R2
134,5X14K8IO8U,gbtree,0.3,0.05,1.2,4,8,reg:squarederror,0,1.0,10.443708,0.571809,9.182725,1.250873,3.513662,1.619213,175.836141,52.429799,132.692157,87.041316,0.518545,0.125096,0.189093,0.111463,8.866087,0.806719,6.398834,0.909766,240.204087,53.988585,300.534713,90.418312,0.459523,0.094961,0.143273,0.036745,11.553430,2.043508,4.273249,1.843870,279.158443,103.332326,198.267526,131.422446,0.521865,0.175526,0.230697,0.175338,12.050853,1.646980,4.213353,1.134222,298.277726,72.609813,206.296498,72.757799,-0.017427,0.342344,0.328807,0.179398,10.840789,1.614398,8.160346,1.685704,391.131379,107.284721,594.694869,161.721196,0.054778,0.133726,0.203987,0.109812,15.902038,3.195682,6.643298,1.860052,593.940002,220.178522,579.110371,291.846989,-0.207729,0.493957,0.425936,0.198426,9.867414,231.732890,0.499978,12.931227,427.783036,-0.056792
341,NDOLFR1ZJT,gbtree,0.3,0.10,1.2,4,8,reg:squarederror,0,1.0,10.403922,0.562595,9.182725,1.250873,3.513662,1.619213,175.836141,52.429799,132.692157,87.041316,0.518545,0.125096,0.189093,0.111463,8.866087,0.806719,6.398834,0.909766,240.204087,53.988585,300.534713,90.418312,0.459523,0.094961,0.143273,0.036745,11.553430,2.043508,4.273249,1.843870,279.158443,103.332326,198.267526,131.422446,0.521865,0.175526,0.230697,0.175338,12.050853,1.646980,4.213353,1.134222,298.277726,72.609813,206.296498,72.757799,-0.017427,0.342344,0.328807,0.179398,10.840789,1.614398,8.160346,1.685704,391.131379,107.284721,594.694869,161.721196,0.054778,0.133726,0.203987,0.109812,15.902038,3.195682,6.643298,1.860052,593.940002,220.178522,579.110371,291.846989,-0.207729,0.493957,0.425936,0.198426,9.867414,231.732890,0.499978,12.931227,427.783036,-0.056792
152,FVBCE8WIBL,gbtree,0.3,0.01,1.2,4,8,reg:squarederror,0,1.0,10.428183,0.560162,9.182725,1.250873,3.513662,1.619213,175.836141,52.429799,132.692157,87.041316,0.518545,0.125096,0.189093,0.111463,8.866087,0.806719,6.398834,0.909766,240.204087,53.988585,300.534713,90.418312,0.459523,0.094961,0.143273,0.036745,11.553430,2.043508,4.273249,1.843870,279.158443,103.332326,198.267526,131.422446,0.521865,0.175526,0.230697,0.175338,12.050853,1.646980,4.213353,1.134222,298.277726,72.609813,206.296498,72.757799,-0.01742

In [22]:
best_params = dict(results_df.iloc[0][[col for col in results_df.columns if 'param_' in col]])
best_params = {key.replace('param_', ''): value for key, value in best_params.items()}
print('Best parameters: {}'.format(best_params))

Best parameters: {'booster': 'gbtree', 'eta': 0.3, 'gamma': 0.05, 'lambda': 1.2, 'max_depth': 4, 'nthread': 8, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 1.0}


## Best model

In [23]:
# Load data
X_train = np.load(os.path.join(RESULTS_PATH, DATA_ID, 'data', 'X_train_{}.npy'.format(DATA_ID)))
X_test = np.load(os.path.join(RESULTS_PATH, DATA_ID, 'data', 'X_test_{}.npy'.format(DATA_ID)))
Y_train = np.load(os.path.join(RESULTS_PATH, DATA_ID, 'data', 'Y_train_{}.npy'.format(DATA_ID)))
Y_test = np.load(os.path.join(RESULTS_PATH, DATA_ID, 'data', 'Y_test_{}.npy'.format(DATA_ID)))

In [29]:
results = defaultdict(list)
tr_time = []
for target in range(Y_train.shape[1]):

    dtrain = xgb.DMatrix(data=X_train, label=Y_train[:, target])
    dtest = xgb.DMatrix(data=X_test, label=Y_test[:, target])

    callbacks = [xgb.callback.EarlyStopping(rounds=5, metric_name='rmse', maximize=False, save_best=True)]
    
    t_start = time.time()
    model = xgb.train(best_params, dtrain, evals=[(dtest, 'rmse')], callbacks=callbacks, verbose_eval=False)
    tr_time.append(time.time() - t_start)
    
    # Save the model
    model.save_model(os.path.join(RESULTS_PATH, DATA_ID, 'XGB_{}'.format(HS_DATE), 'XGB_best_model_{}_{}_{}.joblib'.format(target, HS_DATE, DATA_ID)))
    
    train_preds = model.predict(dtrain)
    test_preds = model.predict(dtest)

    results['Train_MAE'].append(mean_absolute_error(Y_train[:, target], train_preds))
    results['Train_MSE'].append(mean_squared_error(Y_train[:, target], train_preds))
    results['Train_R2'].append(r2_score(Y_train[:, target], train_preds))
    results['Test_MAE'].append(mean_absolute_error(Y_test[:, target], test_preds))
    results['Test_MSE'].append(mean_squared_error(Y_test[:, target], test_preds))
    results['Test_R2'].append(r2_score(Y_test[:, target], test_preds))

print('Training time: {:.4f}'.format(sum(tr_time)))

Training time: 13.5001


In [30]:
# # Display the score for each axis of each force cell
# for subset in ['Train', 'Test']:
#     for f, force in enumerate(['Fx', 'Fy', 'Fz']):
#         for c in range(N_CELLS):
#             for loss in ['MAE', 'MSE', 'R2']:
#                 scores = [results[subset][loss][i + f] for i in range(0, N_CELLS * 3, 3)]
#                 print('{} {}{}{} {}: {:.4f}'.format(subset, force[0], c + 1, force[-1], loss, scores[c]))
# print('\n')

# Display the score mean and standard deviation of each axis
for subset in ['Train', 'Test']:
    for f, force in enumerate(['Fx', 'Fy', 'Fz']):
        for loss in ['MAE', 'MSE', 'R2']:
            scores = [results['_'.join([subset, loss])][i + f] for i in range(0, N_CELLS * 3, 3)]
            print(' '.join([subset, force, loss]) + ': {:.4f} ± {:.4f}'.format(np.mean(scores), np.std(scores)))

Train Fx MAE: 8.3961 ± 3.1096
Train Fx MSE: 141.7054 ± 99.0585
Train Fx R2: 0.5848 ± 0.2275
Train Fy MAE: 7.6546 ± 5.2656
Train Fy MSE: 162.0622 ± 172.9832
Train Fy R2: 0.5838 ± 0.1167
Train Fz MAE: 11.3034 ± 5.3325
Train Fz MSE: 275.4277 ± 219.0808
Train Fz R2: 0.5517 ± 0.3086
Test Fx MAE: 12.0839 ± 4.9598
Test Fx MSE: 289.8891 ± 222.9538
Test Fx R2: 0.4305 ± 0.1368
Test Fy MAE: 10.4137 ± 7.5679
Test Fy MSE: 357.7774 ± 463.0753
Test Fy R2: 0.3818 ± 0.1306
Test Fz MAE: 17.6703 ± 6.5851
Test Fz MSE: 668.5766 ± 703.9588
Test Fz R2: 0.4353 ± 0.1677


In [ ]:
# plt.figure(figsize=(20,15))
# plt.scatter(Y_train[:, 3], Y_train[:, 4], label='true', alpha=0.3)
# plt.scatter(train_preds[:, 3], train_preds[:, 4], label='preds', alpha=0.3)
# plt.legend()
# plt.show()

# plt.figure(figsize=(20,15))
# plt.scatter(Y_train[:100, 3], Y_train[:100, 4], label='true', alpha=0.3)
# plt.scatter(train_preds[:100, 3], train_preds[:100, 4], label='preds', alpha=0.3)
# plt.legend()
# plt.show()

# plt.figure(figsize=(20,15))
# plt.scatter(Y_test[:, 3], Y_test[:, 4], label='true', alpha=0.3)
# plt.scatter(test_preds[:, 3], test_preds[:, 4], label='preds', alpha=0.3)
# plt.legend()
# plt.show()